In [85]:
#Faith May the Lord be with me

import pandas as pd
import numpy as np
from datetime import timedelta

In [86]:
FeatureSetB = pd.read_csv('FeatureSet/AD_48Hours.csv')

In [87]:
def aggregationFunc(tbl):
    Subject_id = tbl['subject_id'].mode()
    GCSEyesAgg = tbl['GCSEyes'].mode()
    GCSMotorAgg = tbl['GCSMotor'].mode()
    GCSVerbalAgg = tbl['GCSVerbal'].mode()
    Bilrubin_LevelAgg = tbl['Bilrubin_Level'].mean()
    BTAgg = tbl['BT'].mean()
    FIO2Agg = tbl['FIO2'].mean()
    HRAgg = tbl['HR'].mean()
    PAO2Agg = tbl['PAO2'].mean()
    Potassium_LevelAgg = tbl['Potassium_Level'].mean()
    SBC_LevelAgg = tbl['SBC_Level'].mean()                 
    SBPAgg = tbl['SBP'].mean()                 
    Sodium_LevelAgg = tbl['Sodium_Level'].mean()           
    SUN_LevelAgg = tbl['SUN_Level'].mean()               
    UrinAgg = tbl['Urin'].mean()   
    WBCC_LevelAgg = tbl['WBCC_Level'].mean()         
    Service_typeAgg = tbl['Service_type'].mode()               
    AgeAgg = tbl['Age'].mode()                       
    MetaStaticAgg = tbl['MetaStatic'].mode()                  
    HematologicAgg = tbl['Hematologic'].mode()                   
    Procedure_typeAgg = tbl['Procedure_type'].mode()                  
    X = pd.DataFrame({
    'subject_id': Subject_id,
    'Procedure_type': Procedure_typeAgg,
    'Bilrubin_Level': Bilrubin_LevelAgg,
    'BT': BTAgg,  
    'FIO2': FIO2Agg,
    'GCSEyes' : GCSEyesAgg,
    'GCSMotor' :  GCSMotorAgg,
    'GCSVerbal' : GCSVerbalAgg,
    'HR' : HRAgg,
    'PAO2': PAO2Agg, 
    'Potassium_Level': Potassium_LevelAgg,
    'SBC_Level': SBC_LevelAgg,
    'SBP': SBPAgg,
    'Service_type': Service_typeAgg,
    'Sodium_Level': Sodium_LevelAgg,         
    'SUN_Level': SUN_LevelAgg,              
    'Urin': UrinAgg,                      
    'WBCC_Level': WBCC_LevelAgg, 
    'Age': AgeAgg,
    'Hematologic' : HematologicAgg,
    'MetaStatic' : MetaStaticAgg,          
    })
    if(len(X.index) != 1):
        for i in range(len(X.index)-1):
            X.drop(i+1, inplace=True)
    return X,AgeAgg

In [88]:
#Handle the NAN Values

FeatureSetAGG = pd.DataFrame()
pd.options.mode.chained_assignment = None  # default='warn'

Y = FeatureSetB
X4,_ = aggregationFunc(Y)
Y['GCSEyes'] = Y['GCSEyes'].fillna(X4.GCSEyes[0])
Y['GCSMotor'] = Y['GCSMotor'].fillna(X4.GCSMotor[0])
Y['GCSVerbal'] = Y['GCSVerbal'].fillna(X4.GCSVerbal[0])
Y['Bilrubin_Level'] = Y['Bilrubin_Level'].fillna(X4.Bilrubin_Level[0])
Y['BT'] = Y['BT'].fillna(X4.BT[0])
Y['FIO2'] = Y['FIO2'].fillna(X4.FIO2[0])
Y['HR'] = Y['HR'].fillna(X4.HR[0])
Y['PAO2'] = Y['PAO2'].fillna(X4.PAO2[0])
Y['Potassium_Level'] = Y['Potassium_Level'].fillna(X4.Potassium_Level[0])
Y['SBC_Level'] = Y['SBC_Level'].fillna(X4.SBC_Level[0])      
Y['SBP'] = Y['SBP'].fillna(X4.SBP[0])        
Y['Sodium_Level'] = Y['Sodium_Level'].fillna(X4.Sodium_Level[0])        
Y['SUN_Level'] = Y['SUN_Level'].fillna(X4.SUN_Level[0])       
Y['Urin'] = Y['Urin'].fillna(X4.Urin[0])
Y['WBCC_Level'] = Y['WBCC_Level'].fillna(X4.WBCC_Level[0])        
Y['Service_type'] = Y['Service_type'].fillna(X4.Service_type[0])             
Y['Age'] = Y['Age'].fillna(X4.Age[0])
Y['MetaStatic'] = Y['MetaStatic'].fillna(X4.MetaStatic[0])              
Y['Hematologic'] =  Y['Hematologic'].fillna(X4.Hematologic[0])               
Y['Procedure_type'] = Y['Procedure_type'].fillna(X4.Procedure_type[0])
FeatureSetAGG = pd.concat([FeatureSetAGG,Y])

In [89]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSetAGG.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)
Subject_id.reset_index(drop=True,inplace=True) 

In [90]:
#Finalize the windowing algorithm
# 1 , 2 , 5 , 7, 10 , 24, 48

NumOfWindows = 48
TotalWindows = 48
FeatureSet_L1 = pd.DataFrame()

for L1 in Subject_id:
    FeatureSet_L2 = pd.DataFrame()
    IndexPerWindow = round(48/NumOfWindows)
    Y = FeatureSetAGG.loc[FeatureSetAGG['subject_id'] == L1]
    Y_Index = 0
    IndexUsed = 0
    for L2 in range(NumOfWindows):
        Y_Agg = Y[Y_Index:Y_Index+IndexPerWindow]
        Y_Agg,age = aggregationFunc(Y_Agg)
        age = int(age)
        IndexUsed = IndexUsed+IndexPerWindow
        Y_Index = Y_Index+IndexPerWindow
        if (L2 == NumOfWindows-2 ):
            IndexPerWindow = 48-IndexUsed
        if (age > 15):
            FeatureSet_L2 = pd.concat([FeatureSet_L2,Y_Agg])
    FeatureSet_L1 = pd.concat([FeatureSet_L1 ,FeatureSet_L2])

In [91]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSet_L1.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)

In [92]:
result = []
for X in Subject_id:
    Feature = pd.DataFrame()
    Y = FeatureSet_L1.loc[FeatureSet_L1['subject_id'] == X]
    Z = Y.shape
    Rows = Z[0]
    result.append((X, Rows))

In [93]:
result_df = pd.DataFrame(result,columns =['subject_id','Rows'])

In [94]:
results = result_df.drop(result_df.index[result_df['Rows'] == 5])
results.reset_index(drop=True,inplace=True)

In [95]:
X1 = FeatureSet_L1.subject_id.isin(results.subject_id)
Test = FeatureSet_L1[X1]

In [96]:
FeatureSet_L1.to_csv('MIMIC_Windows/FSB_48Ws.csv', encoding='utf-8')